# Workflow
## Step 1: Gather the Test Inputs
### The test session with all the pictures and coordinates
### The global coordinates of the device
## Step 2: Check for relevant Reference data
### use the global coordinates to find all the reference data that is georeferenced close enough (GPS presision)
## Step 3: 2D Check
### Compare all the test images against all the reference images
### Find which session has the highest match rate
### Find which Image has the highest match rate
### Calculate the transformation between the two images
### calculate the inverse transformation to give the test data a Reference global position
## Step 4: 3D Check
### compare the test mesh against relevant point clouds
### Compare the test mesh against BIM models
### Perform a CCP for the final alignment
## Step 5: Choosing final Position
### Use the different results from all the methods to come to a best position
### Send the Position and rotation back to the device



## Step 1: Gather the Test Inputs

In [39]:
from cv2 import transform
import numpy as np

# These will be received by the server at a later stage in development
testGlobalPosition = (0,0,0) # a tuple containing the global coordinates received by the locationSender app
testGlobalAccuracy  = 20 # the global accuracy of the Coordinates sent by the locationSender app
testSessionDirLocation = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-07 House Trekweg/RAW Data/Hololens/Sessions/session-2021-10-12 14-28-27" # The location of the test session
# Reference Data
referenceSessionDirLocation = "/Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-07 House Trekweg/RAW Data/Android"

## Step 2: check for relevant Reference data

In [40]:
from findBestSessions import FindBestSessions
closeEnoughSessions = FindBestSessions(referenceSessionDirLocation, np.array(testGlobalPosition), testGlobalAccuracy)

Found Session data: /Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-07 House Trekweg/RAW Data/Android/sessions/session-2021-11-04 16-17-08/SessionData.json
Reference global position: [-1 -7  0]
Distance from coordinate: 7.0710678118654755
session-2021-11-04 16-17-08 : is close enough
 
Found Session data: /Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-07 House Trekweg/RAW Data/Android/sessions/session-2021-11-04 16-41-00/SessionData.json
Reference global position: [0 0 0]
Distance from coordinate: 0.0
session-2021-11-04 16-41-00 : is close enough
 
Found Session data: /Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-07 House Trekweg/RAW Data/Android/sessions/session-2021-11-04 16-46-29/SessionData.json
Reference global position: [ 3  4 -8]
Distance from coordinate: 9.433981132056603
session-2021-11-04 16-46-29 : is close enough
 
Found Session data: /Volumes/GeomaticsProjects1/Projects/2025-03 Project F

## Step 3: 2D Check

In [64]:
from compareImageSession import CompareImageSession
from imageTransform import ImageTransform
from bestResult import BestResult

testSessionJsonPath = testSessionDirLocation + "/SessionData.json"
bestResult = BestResult(0,0,0,0,0)
bestRefImage = 0
bestTestImage = 0
bestSession = 0

for referenceSession in closeEnoughSessions:
    result,testImage,refImage = CompareImageSession(testSessionJsonPath, referenceSession)
    if(result.matchAmount > bestResult.matchAmount):
        bestResult = result
        bestRefImage = refImage
        bestTestImage = testImage
        bestSession = referenceSession



3 reference Images found in: /Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-07 House Trekweg/RAW Data/Android/sessions/session-2021-11-04 16-17-08
4 test Images found in: /Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-07 House Trekweg/RAW Data/Hololens/Sessions/session-2021-10-12 14-28-27
1/12 checks complete
2/12 checks complete
3/12 checks complete
4/12 checks complete
5/12 checks complete
6/12 checks complete
7/12 checks complete
8/12 checks complete
9/12 checks complete
10/12 checks complete
11/12 checks complete
12/12 checks complete
This is the best image with 75 match amount
Estimated homography : 
 [[-1.18608206e-01  1.18793909e-02  3.43715839e+02]
 [-7.80889649e-02 -1.59215077e-02  2.54448397e+02]
 [-3.05678207e-04 -4.85877130e-05  1.00000000e+00]]
6 reference Images found in: /Volumes/GeomaticsProjects1/Projects/2025-03 Project FWO SB Jelle/7.Data/21-07 House Trekweg/RAW Data/Android/sessions/session-2021-11-04 16-41-0

### Calculate the transformation

In [63]:
import importlib
import getGlobalPositionOffset 
importlib.reload(getGlobalPositionOffset)
from getGlobalPositionOffset import GetGlobalPositionOffset
from transform import Transform
import json

refSessionJson = json.load(open(bestSession,))
bestGlobalPosition = Transform(refSessionJson["sessionId"],refSessionJson["globalPosition"], refSessionJson["globalRotation"], [1,1,1])

newGlobalPosition =  GetGlobalPositionOffset(bestTestImage, bestRefImage, bestGlobalPosition, bestResult.transMatrix)

print(newGlobalPosition)

The reference Image Global position: [-1.5129 -6.915   0.0375]
The reference Image Global rotation: quaternion(0.9291, -0.0294, -0.3682, -0.0182)
The transformationMatrix: 
[[-1.18608206e-01  1.18793909e-02  3.43715839e+02]
 [-7.80889649e-02 -1.59215077e-02  2.54448397e+02]
 [-3.05678207e-04 -4.85877130e-05  1.00000000e+00]]
The test Image local position: [ 0.0581 -0.1502  0.9954]
The test Image Global position: [12.98664032  9.77005289  0.03829844]
The Calculated test Global offset:[12.92854032  9.92025289 -0.95710156]
[12.92854032  9.92025289 -0.95710156]
